# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import platform
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [2]:
# 실험 번호
TEST_NUM = "t5-base-01"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + f"output-{TEST_NUM}/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_kkh_new.csv"
VALID_PATH = DATA_PATH + "dev_kkh_new.csv"
TEST_PATH = DATA_PATH + "test.csv"

# 허깅페이스 관련
# HUGGING_MODEL = "digit82/kobart-summarization"
# HUGGING_MODEL = "EbanLee/kobart-summary-v3"
# HUGGING_MODEL = "gogamza/kobart-base-v2"
# HUGGING_MODEL = "gogamza/kobart-summarization"
# HUGGING_MODEL = "alaggung/bart-r3f"
# HUGGING_MODEL = "NLPBada/kobart-chat-persona-extraction-v2"
# HUGGING_MODEL = "lcw99/t5-large-korean-text-summary"
HUGGING_MODEL = "eenzeenee/t5-base-korean-summarization"

# 완디비 관련
WANDB_ENTITY = 'helpot'
WANDB_PROJECT_NAME = 'bootcamp-upstage-nlp'
WANDB_RUN_NAME = TEST_NUM
WANDB_LOG = "end" # end or checkpoint
WANDB_WATCH = "false" # true or false


In [3]:
def manage_output_folder(output_path):
    if isinstance(output_path, tuple):
        output_path = output_path[0]
    if not isinstance(output_path, str):
        raise ValueError("output_path should be a string")
    if output_path.endswith('/') or output_path.endswith('\\'):
        output_path = output_path[:-1]
    
    if os.path.exists(output_path):
        # 폴더가 존재할 때 폴더가 비어 있는지 확인
        if not os.listdir(output_path):
            # 폴더가 비어 있으면 그대로 사용
            print(f"Folder exists and is empty. Using the existing folder: {output_path}")
        else:
            # 폴더가 비어 있지 않으면 "_backup"을 붙여 이름을 변경
            backup_path = output_path + "_backup"
            print(f"Folder already exists and is not empty. Renaming to: {backup_path}")
            shutil.move(output_path, backup_path)
            # 새로운 폴더 생성
            os.makedirs(output_path)
            print(f"New folder created: {output_path}")
    else:
        # 폴더가 존재하지 않으면 새로 생성
        os.makedirs(output_path)
        print(f"New folder created: {output_path}")

# 함수 호출
manage_output_folder(OUTPUT_PATH)
manage_output_folder(CHECKPOINT_PATH)
manage_output_folder(PREDICTION_PATH)

Folder exists and is empty. Using the existing folder: /kkh/output
Folder exists and is empty. Using the existing folder: /kkh/checkpoint
Folder exists and is empty. Using the existing folder: /kkh/prediction


In [4]:
tokenizer = AutoTokenizer.from_pretrained(HUGGING_MODEL)

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
loaded_config = {
    "general": {
        "data_path": DATA_PATH,
        "output_dir": OUTPUT_PATH,
        "model_name": HUGGING_MODEL,
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 204,
        # "special_tokens": ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']
        "special_tokens": ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', '#PhoneNumber#', '#SSN#'],
        "person_tokens": ['#Person#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 50,  # Seq2SeqTrainingArguments 기반 설정
        "learning_rate": 1e-5,  # Seq2SeqTrainingArguments 기반 설정
        "per_device_train_batch_size": 4,  # Seq2SeqTrainingArguments 기반 설정
        "per_device_eval_batch_size": 4,  # Seq2SeqTrainingArguments 기반 설정
        "auto_find_batch_size": False,  # Seq2SeqTrainingArguments 기반 설정
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,  # Seq2SeqTrainingArguments 기반 설정
        "lr_scheduler_type": 'linear',  # Seq2SeqTrainingArguments 기반 설정
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 20,
        "fp16": True,  # Seq2SeqTrainingArguments 기반 설정
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": LOG_PATH,
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 200,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"
    },
    "wandb": {
        "entity": WANDB_ENTITY,
        "project": WANDB_PROJECT_NAME,
        "name": WANDB_RUN_NAME
    },
    "inference": {
        "ckt_path": CHECKPOINT_PATH,
        "result_path": PREDICTION_PATH,
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": [f"{tokenizer.pad_token}"]
    }
}


In [6]:
train_df = pd.read_csv(os.path.join(TRAIN_PATH))
val_df = pd.read_csv(os.path.join(VALID_PATH))

## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [7]:
# 데이터 전처리를 위한 클래스: 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self, prefix: str) -> None:
        self.prefix = prefix  # T5 모델에서 사용할 태스크 프리픽스 (예: "summarize: ")

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            test_df = df[['fname', 'dialogue']]
            return test_df

    # T5 모델의 입력 형태에 맞게 전처리를 진행합니다.
    def make_input(self, dataset, is_test=False):
        if is_test:
            # 테스트 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = [""] * len(dataset['dialogue'])  # T5 모델에서는 테스트 시 디코더 입력이 필요하지 않습니다.
            return encoder_input.tolist(), decoder_input
        else:
            # 학습 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가하고, 출력 텍스트로 학습합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = dataset['summary'].apply(lambda x: str(x))  # T5 모델에서는 디코더 입력을 별도로 준비하지 않습니다.
            decoder_output = dataset['summary'].apply(lambda x: str(x))  # 목표 요약 문장을 디코더 출력으로 사용합니다.
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [8]:
from torch.utils.data import Dataset

# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, length):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.length


In [9]:
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    # 인코더 입력 및 출력 데이터를 준비합니다.
    encoder_input_train, _, decoder_output_train = preprocessor.make_input(train_df)
    encoder_input_val, _, decoder_output_val = preprocessor.make_input(val_df)
    print('-' * 10, 'Load data complete', '-' * 10)

    # 인코더 입력을 토크나이저로 토큰화합니다.
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                                         add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)

    # 디코더 출력을 토크나이저로 토큰화합니다 (T5 모델에서 labels로 사용).
    tokenized_labels = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                                 add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    # 학습 데이터셋 생성
    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_labels, len(encoder_input_train))

    # 검증 데이터의 인코더 입력을 토크나이저로 토큰화합니다.
    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                                             add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)

    # 검증 데이터의 디코더 출력을 토크나이저로 토큰화합니다 (T5 모델에서 labels로 사용).
    val_tokenized_labels = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                                     add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    # 검증 데이터셋 생성
    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_labels, len(encoder_input_val))

    print('-' * 10, 'Make dataset complete', '-' * 10)
    return train_inputs_dataset, val_inputs_dataset


## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [10]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [11]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )
    os.environ["WANDB_LOG_MODEL"] = WANDB_LOG
    os.environ["WANDB_WATCH"] = WANDB_WATCH

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [12]:
from transformers import AutoTokenizer, AutoConfig, T5ForConditionalGeneration

# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)

    # 모델 이름 가져오기
    model_name = config['general']['model_name']

    # T5 모델 설정 로드
    t5_config = AutoConfig.from_pretrained(model_name)
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # T5 모델 로드
    generate_model = T5ForConditionalGeneration.from_pretrained(model_name, config=t5_config)

    # 스페셜 토큰 추가
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.add_tokens(config['tokenizer']['person_tokens'])

    # 추가된 토큰 수에 맞춰 모델의 토큰 임베딩 크기 재조정
    generate_model.resize_token_embeddings(len(tokenizer))

    # 모델을 지정된 장치로 이동
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model, tokenizer


## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [13]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
    print('-' * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-' * 10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    # T5 모델은 bos_token과 eos_token을 사용하지 않으므로, prefix를 전달합니다.
    preprocessor = Preprocess(prefix=config['tokenizer'].get('prefix', 'summarize: ')) 
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()


In [14]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.3.1+cu121
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------


/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5Config {
  "_name_or_path": "eenzeenee/t5-base-korean-summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 50374
}

---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'eos_token': '</s>', 'unk_token': '<pad>', 'pad_token': '<pad>',

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: helpotcreator (helpot). Use `wandb login --relogin` to force relogin


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2#는 숨쉬기가 힘들고 천식 증상이 있습니다. #Person1#는 #Person2#를 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 것입니다.</s>                                                                                                                           
GOLD: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다. </s>                                                                                                       
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1#과 지미는 나중에 운동하러 가기로 결정한다. 그들은 금요일에 다리와 팔을 하기로 결정한다.</s>                                                                                                                                   
GOLD: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.</s>                      

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2#는 숨쉬기가 힘들고 운동을 할 때 많이 느낍니다. #Person1#는 #Person2#를 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 것입니다.</s>                                                                                                  
GOLD: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다. </s>                                                                                                       
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1#과 지미는 나중에 운동하러 가기로 결정한다. 그들은 금요일에 다리와 팔을 하기로 결정한다.</s>                                                                                                              
GOLD: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.</s>                                                                

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}


eval/loss,█▄▂▂▁▁▁▁▁▁▁▁
eval/rouge-1,▁▃▇▇▇▇██████
eval/rouge-2,▁▂▅▆▆▇▇▇████
eval/rouge-l,▁▃▇▇▇▇██████
eval/runtime,█▅▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁▃█▇█▇▇▇█▇█▇
eval/steps_per_second,▁▃█▇█▇▇▇█▇█▇
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/grad_norm,█▁▂▂▂▂▁▁▁▁▁▂
train/learning_rate,▁▃▅▆███▇▇▇▇▇


## 4. 모델 추론하기

In [26]:
# 이곳에 내가 사용할 wandb config 설정
SCORE_L = "37344"
loaded_config['inference']['ckt_path'] = f"{OUTPUT_PATH}checkpoint-{SCORE_L}/"

- test data를 사용하여 모델의 성능을 확인합니다.

In [27]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):
    # 테스트 데이터 경로 설정
    test_file_path = os.path.join(config['general']['data_path'], 'test.csv')

    # 테스트 데이터셋 준비
    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-' * 150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-' * 150)

    # 인코더 입력 준비 (T5 모델에서는 디코더 입력을 따로 준비하지 않음)
    encoder_input_test, _ = preprocessor.make_input(test_data, is_test=True)
    print('-' * 10, 'Load data complete', '-' * 10,)

    # 인코더 입력을 토크나이저로 토큰화합니다.
    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config['tokenizer']['encoder_max_len'],
        return_token_type_ids=False,
    )

    # 테스트 데이터셋 생성
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-' * 10, 'Make dataset complete', '-' * 10,)

    return test_data, test_encoder_inputs_dataset

In [28]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    # 모델 이름과 체크포인트 경로 설정
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 스페셜 토큰 추가
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.add_tokens(config['tokenizer']['person_tokens'])

    # T5 모델 로드
    generate_model = T5ForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    
    # 모델을 지정한 장치로 이동
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model, tokenizer


In [29]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader

# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    # 사용할 장치를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 모델과 토크나이저를 로드합니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 데이터 경로와 전처리 객체를 정의합니다.
    data_path = config['general']['data_path']
    preprocessor = Preprocess(prefix=config['tokenizer'].get('prefix', 'summarize: '))

    # 테스트 데이터셋을 준비합니다.
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []

    # 추론을 수행합니다.
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            input_ids = item['input_ids'].to(device)
            
            generated_ids = generate_model.generate(
                input_ids=input_ids,
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                early_stopping=config['inference']['early_stopping'],
                max_length=config['inference']['generate_max_length'],
                num_beams=config['inference']['num_beams'],
            )
            
            # 요약문 디코딩 및 저장
            for ids in generated_ids:
                # 스페셜 토큰을 유지하기 위해 skip_special_tokens=False로 설정합니다.
                result = tokenizer.decode(ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

                # 불필요한 스페셜 토큰 제거
                result = result.replace('<pad>', '').replace('</s>', '').strip()
                summary.append(result)

    # 결과를 데이터프레임으로 변환하고 파일로 저장
    output = pd.DataFrame({"fname": test_data['fname'], "summary": summary})
    result_path = config['inference']['result_path']
    
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    
    output.to_csv(os.path.join(result_path, f"{TEST_NUM}-{SCORE_L}.csv"), index=False)

    return output


In [30]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.3.1+cu121
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------


/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 

100%|██████████| 16/16 [01:41<00:00,  6.34s/it]


In [31]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,"#Person1#은 모든 직원에게 내부 메모를 전달하고, 즉시 효력을 발휘하여 사무..."
1,test_1,#Person2#는 교통 체증에 걸렸다. #Person1#은 #Person2#에게 ...
2,test_2,#Person1#은 케이트에게 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신...
3,test_3,#Person1#은 브라이언의 생일을 축하하기 위해 파티에 참석한다. 그들은 함께 ...
4,test_4,"#Person2#는 #Person1#에게 올림픽 스타디움이 6월에 완공될 예정이며,..."
...,...,...
494,test_495,찰리는 잭에게 집에 와서 자신의 캐릭터를 만드는 비디오 게임을 하자고 제안한다.
495,test_496,#Person2#는 #Person1#에게 컨트리 음악에 관심을 가지게 된 계기와 라...
496,test_497,#Person1#은 옷을 깨끗하게 하고 싶어서 앨리스에게 기계를 어떻게 사용하는지 ...
497,test_498,스티브는 매튜에게 최근에 살 곳을 찾고 있다고 말한다. 그의 이웃인 다우 부인이 아...
